# Inventory Segmentation using ABC–XYZ Analysis

This notebook segments products based on revenue contribution (ABC)
and demand variability (XYZ) to support inventory prioritization
and supply chain decision-making.


In [1]:
import pandas as pd
import numpy as np

orders = pd.read_csv("../data/processed/orders_features.csv")
orders["order_date"] = pd.to_datetime(orders["order_date"])


In [2]:
product_sales = (
    orders
    .groupby("Product Name")["net_sales"]
    .sum()
    .sort_values(ascending=False)
)


In [3]:
cumulative_sales_pct = product_sales.cumsum() / product_sales.sum()


In [4]:
abc_segment = pd.cut(
    cumulative_sales_pct,
    bins=[0, 0.80, 0.95, 1.00],
    labels=["A", "B", "C"]
)


In [5]:
abc_df = pd.DataFrame({
    "Product Name": product_sales.index,
    "total_net_sales": product_sales.values,
    "cumulative_sales_pct": cumulative_sales_pct.values,
    "ABC_segment": abc_segment.values
})


In [6]:
orders["year_month"] = orders["order_date"].dt.to_period("M")

monthly_demand = (
    orders
    .groupby(["Product Name", "year_month"])["Order Quantity"]
    .sum()
    .reset_index()
)


In [7]:
demand_stats = (
    monthly_demand
    .groupby("Product Name")["Order Quantity"]
    .agg(["mean", "std"])
    .reset_index()
)


In [8]:
demand_stats["cv"] = demand_stats["std"] / demand_stats["mean"]


In [9]:
def xyz_label(cv):
    if cv < 0.25:
        return "X"   # Stable demand
    elif cv <= 0.50:
        return "Y"   # Medium variability
    else:
        return "Z"   # High variability

demand_stats["XYZ_segment"] = demand_stats["cv"].apply(xyz_label)


In [10]:
inventory_segmentation = abc_df.merge(
    demand_stats[["Product Name", "XYZ_segment"]],
    on="Product Name",
    how="left"
)


In [12]:
inventory_segmentation["ABC_segment"] = inventory_segmentation["ABC_segment"].astype(str)
inventory_segmentation["XYZ_segment"] = inventory_segmentation["XYZ_segment"].astype(str)


In [13]:
inventory_segmentation["ABC_XYZ_segment"] = (
    inventory_segmentation["ABC_segment"] +
    inventory_segmentation["XYZ_segment"]
)


In [14]:
inventory_segmentation[["ABC_segment", "XYZ_segment", "ABC_XYZ_segment"]].head()


,ABC_segment,XYZ_segment,ABC_XYZ_segment
0,A,X,AX
1,A,X,AX
2,A,Y,AY
3,A,X,AX
4,A,Y,AY


In [15]:
inventory_segmentation["ABC_XYZ_segment"].value_counts()


ABC_XYZ_segment
CZ    73
CY    18
BZ    11
AX     4
AY     2
BY     2
CX     2
BX     1
Name: count, dtype: int64

In [16]:
inventory_segmentation.to_csv(
    "../data/processed/inventory_segmentation_abc_xyz.csv",
    index=False
)


## ABC–XYZ Segmentation Summary

- A-segment products contribute the majority of revenue.
- X-segment products show stable demand patterns.
- XA and YA products represent core revenue drivers and require high service levels.
- Z-segment products exhibit volatile demand and should be managed with caution.
